In [19]:
import pandas as pd
import numpy as np
from binance.client import Client
client = Client()

In [20]:
def getdata(symbol,start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'4h',start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time',inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

In [21]:
df = getdata('BTCUSDT','2021-01-01')
df

,Open,High,Low,Close,Volume
Time,,,,,
2021-01-01 00:00:00,28923.63,29470.00,28690.17,29278.40,11560.456553
2021-01-01 04:00:00,29278.41,29395.00,28806.54,29092.83,7308.910274
2021-01-01 08:00:00,29092.84,29402.57,28872.24,29313.49,8283.705319
2021-01-01 12:00:00,29313.49,29600.00,29030.14,29188.67,11794.949515
2021-01-01 16:00:00,29188.67,29360.00,28624.57,29029.04,9850.965345
...,...,...,...,...,...
2022-12-15 00:00:00,17804.01,17854.82,17561.00,17729.56,34212.626710
2022-12-15 04:00:00,17729.57,17746.00,17681.30,17691.45,18037.652200
2022-12-15 08:00:00,17690.84,17726.00,17610.22,17720.33,26381.545130


In [22]:
import pandas_ta as ta
#df.ta.indicators() shows all indicators in library
#hep(ta.atr) explains indicator,change 'atr' for its name
def add_indicators(df, ema_value,sma_value):
    df['EMA_13'] = df.ta.ema(length=ema_value)
    df['SMA_30'] = df.ta.sma(length=sma_value)
    df.dropna(inplace=True)
    return df
df = add_indicators(df,13,30)

In [23]:
df['Condition'] = (df.EMA_13 > df.SMA_30)

In [24]:
df['Buy'] = (df.Condition) & (df.Condition.shift(1) == False)

In [25]:
# shifts open to one row below // you buy or sell open next candle, just to make stuff easier tohave in the same row
df['shifted_open'] = df.Open.shift(-1)

In [26]:
in_position = False

buy_dates,sell_dates = [],[]
buy_prices,sell_prices = [],[]

for index,row in df.iterrows():
    if not in_position and row.Buy == True:
        buyprice = row.shifted_open
        buy_dates.append(index)
        buy_prices.append(buyprice)
        in_position = True
    if in_position:
        # STOP LOSS
        if row.Low < buyprice * 0.99:
            sellprice = buyprice * 0.99
            sell_prices.append(sellprice)
            sell_dates.append(index)
            in_position = False
        # TAKE PROFIT
        elif row.High > buyprice * 1.01:
            sellprice = buyprice * 1.01
            sell_dates.append(index)
            sell_prices.append(sellprice)
            

In [27]:
profits = pd.Series([(sell-buy)/buy for sell, buy in zip(sell_prices, buy_prices)])
profits

0    -0.010000
1    -0.010000
2    -0.010000
3    -0.010000
4    -0.010000
5    -0.010000
6    -0.010000
7    -0.010000
8     0.010000
9     0.100486
10    0.032683
11    0.024949
12    0.540179
13    0.610921
14    0.595097
15    0.510852
16    0.691434
17    0.714035
18    0.655879
19    0.739343
20    0.846381
21    0.935601
22    0.938390
23    0.969343
24    0.984895
25    0.942284
26    0.883960
27    1.859910
28    1.972988
29    1.860025
30    1.548444
31    1.528756
32    1.456726
33    1.738353
34    1.902040
35    1.931410
36    1.857454
37    2.056437
38    2.086921
39    1.933190
40    2.008138
41    1.974958
42    2.042741
43    2.055326
44    2.072114
45    2.473852
46    2.497578
47    2.550798
48    2.499758
49    2.427175
50    2.379570
dtype: float64